In [1]:
import os
import json
import gzip
import pandas as pd

from dotenv import load_dotenv,find_dotenv


load_dotenv(find_dotenv('config.env'))


True

In [2]:
from langchain.llms import OpenAI

import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'
save_dir = '/home/yumk/Desktop/trained_qlora/llama2hf'

In [4]:
model = AutoModelForCausalLM.from_pretrained(save_dir, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [8]:

text = "The most wanted thing by 20's male is"
#temparature = 0.7

inputs = tokenizer(text, return_tensors="pt").to('cpu')
outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7)
response=tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

The most wanted thing by 20's male is a 40's female.
Submitted by jimmythekid on Oct 14, 13 at 10:54am
Haha, nice. I'm sure they'll be lining up for that one.
Submitted by pizza411 on Oct 14, 13 at 11:00am
I think you mean "a 40's female".
Sub


In [12]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [16]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

Found model file at  /home/yumk/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


In [18]:
q = "Booths can be in mant different variations to suit other people's needs"
docs = vectorstore.similarity_search(q)
len(docs)

4

In [19]:
docs[0]

Document(page_content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"})